In [1]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: renhehuang0723 (renhehuang0723-taiwan-mobile) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
from huggingface_hub import login
login()

In [3]:
!git clone https://github.com/Edwarddev0723/bert-base-chinese-traditional-classifier.git

Cloning into 'bert-base-chinese-traditional-classifier'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 144 (delta 59), reused 98 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 407.67 KiB | 1.68 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [4]:
%cd /content/bert-base-chinese-traditional-classifier

/content/bert-base-chinese-traditional-classifier


In [6]:
# 刪掉所有本機路徑列

!pip install -r requirements.txt

  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached fastapi_cli-0.0.7-py3-none-any.whl.metadata (6.2 kB)
  Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached onnxruntime-1.22.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached OpenCC-1.1.9-cp311-cp311-manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached pyqt6-6.9.1-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (2.1 kB)
  Using cached PySide6-6.9.1-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (5.3 kB)
  Using cached rembg-2.0.66-py3-none-any.whl.metadata (21 kB)
  Using cached transformers-4.53.0-py3-none-any.whl.metadata (39 kB)
  Using cached ultralytics-8.3.161-py3-none-any.whl.metadata (37 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
INFO: pip is looki

In [1]:
%cd /content/bert-base-chinese-traditional-classifier/src

/content/bert-base-chinese-traditional-classifier/src


In [2]:
!python data_prepare.py \
    --rows 2000 \
    --out_dir data_cache_v2 \
    --min_zh_ratio 0.2

{
  "seed": 42,
  "max_tok": 256,
  "buffer_size": 50000,
  "batch_size": 5000,
  "min_len": 10,
  "min_zh_ratio": 0.2,
  "rows": 2000,
  "out_dir": "data_cache_v2",
  "out_name": "fineweb_mix_2000.parquet",
  "tokenizer_name": "ckiplab/bert-base-chinese",
  "hf_endpoint": "https://hf-mirror.com",
  "sources": [
    {
      "dataset": "voidful/fineweb-zhtw",
      "split": "train",
      "mode": "norm",
      "label": 1,
      "key": "text"
    },
    {
      "dataset": "austenjs/ClueCorpusSmallDataset",
      "split": "train",
      "mode": "simp2trad",
      "label": 0,
      "key": "text"
    }
  ],
  "num_proc": 4
}
README.md: 2.61kB [00:00, 8.69MB/s]
Resolving data files: 100% 322/322 [00:00<00:00, 2631.77it/s]
Resolving data files: 100% 322/322 [00:00<00:00, 2464.82it/s]
stream fineweb-zhtw: 100%|█████████████████| 1000/1000 [00:07<00:00, 129.42it/s]
Map (num_proc=4): 100% 1000/1000 [00:01<00:00, 984.34 examples/s]
tokenizer_config.json: 100% 174/174 [00:00<00:00, 716kB/s]
config

In [ ]:

from datasets import load_dataset, Dataset
ds = load_dataset("parquet", data_files="data_cache_v2/fineweb_mix_2000.parquet", split="train")
ds# 應該含 'text' 或你指定的欄位



Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 1938
})

In [3]:
!python tokenizer_util.py \
  --input data_cache_v2/fineweb_mix_2000.parquet \
  --out_dir data_cache_v2 \
  --model ckiplab/bert-base-chinese \
  --max_len 256 --stride 128 --test_size 0.2

Map (num_proc=2): 100% 1550/1550 [00:03<00:00, 410.78 examples/s]
Map (num_proc=2): 100% 388/388 [00:00<00:00, 389.28 examples/s]
Saving the dataset (1/1 shards): 100% 1550/1550 [00:00<00:00, 84040.06 examples/s]
Saving the dataset (1/1 shards): 100% 388/388 [00:00<00:00, 63517.82 examples/s]
✓ Encoded dataset saved to /content/bert-base-chinese-traditional-classifier/src/data_cache_v2


In [ ]:
!pip install --no-cache-dir torch torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 280.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 MB 145.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 242.7 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.3.0
    Uninstalling triton-3.3.0:
      Successfully uninstalled triton-3.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.15 requires torchvision, which is not installed.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.


In [4]:
!python train.py -d data_cache_v2 -o ckpt --rows 5000 \
  --project myproj --run debug-5k

2025-07-02 07:23:00.726609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751440980.893264    4814 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751440980.940493    4814 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-02 07:23:01.081976: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Currently logged in as: renhehuang0723 (renhehuang0723-taiwan-mobile) to https://api.wandb.ai. Use `wandb logi